In [5]:
import loader
from time import time
import numpy as np
import os
import sys

In [23]:
def rbm(dataset, num_hidden, learn_rate, epochs, batchsize):
    num_visible = dataset.shape[1]
    num_examples = dataset.shape[0]
    print("Training RBM with", num_visible, "visible units,", 
          num_hidden, "hidden units,", num_examples, "examples and", 
          epochs, "epochs...")

    start_time = time()

    batches = num_examples // batchsize
   
    w = 0.1 * np.random.randn(num_visible, num_hidden)
    a = np.zeros((1, num_visible))
    b = -4.0 * np.ones((1, num_hidden))

    w_inc = np.zeros((num_visible, num_hidden))
    a_inc = np.zeros((1, num_visible))
    b_inc = np.zeros((1, num_hidden))

    for epoch in range(epochs):
        error = 0
        for batch in range(batches):
         #### --- Positive phase of contrastive divergence --- ####

         # get next batch of data
            v0 = dataset[int(batch*batchsize):int((batch+1)*batchsize)]

         # in this matrix, m[i,j] is prob h[j] = 1 given example v[i]
         # dims: [num_ex] x [num_hidden]
            prob_h0 = logistic(v0, w, b)
    
         # sample the states of hidden units based on prob_h0
            h0 = prob_h0 > np.random.rand(batchsize, num_hidden)

         # positive phase products
            vh0 = np.dot(v0.T, prob_h0)

         # activation values needed to update biases
            poshidact = np.sum(prob_h0, axis=0)
            posvisact = np.sum(v0, axis=0)

         #### --- Negative phase of contrastive divergence --- ####

         # reconstruct the data by sampling the visible states from hidden states
            v1 = logistic(h0, w.T, a)

         # sample hidden states from visible states
            prob_h1 = logistic(v1, w, b)

         #negative phase products
            vh1 = np.dot(v1.T, prob_h1)

         # activation values needed to update biases
            neghidact = np.sum(prob_h1, axis=0)
            negvisact = np.sum(v1, axis=0)

         #### --- Updating the weights --- ####

         # set momentum as per Hinton's practical guide to training RBMs
            m = 0.5 if epoch > 5 else 0.9

         # update the weights
            w_inc = w_inc * m + (learn_rate/batchsize) * (vh0 - vh1)
            a_inc = a_inc * m + (learn_rate/batchsize) * (posvisact - negvisact)
            b_inc = b_inc * m + (learn_rate/batchsize) * (poshidact - neghidact)
            
            a += a_inc
            b += b_inc
            w += w_inc

            error += np.sum((v0 - v1) ** 2)

        print("Epoch %s completed. Reconstruction error is %0.2f. Time elapsed (sec): %0.2f" 
            % (epoch + 1, error, time() - start_time))

    print ("Training completed.\nTotal training time (sec): %0.2f \n" % (time() - start_time))
    return w, a, b


def logistic(x,w,b):
    xw = np.dot(x, w)
    replicated_b = np.tile(b, (x.shape[0], 1))

    return 1.0 / (1 + np.exp(- xw - b))


def reconstruct(v0, w, a, b):
    num_hidden = w.shape[1]
    prob_h0 = logistic(v0, w, b)
    h0 = prob_h0 > np.random.rand(1, num_hidden)

    return logistic(h0, w.T, a)

def sample_hidden(v0,w,b):
    num_hidden = w.shape[1]
    return logistic(v0, w, b)


def save_weights(w, a, b, directory, n_examples, num_hidden):
    if not os.path.exists(directory):
        os.makedirs(directory)

    w_name = directory + os.sep + "w_v" + str(n_examples) + "_h" + str(num_hidden)
    np.save(w_name, w)
    a_name = directory + os.sep + "a_v" + str(n_examples) + "_h" + str(num_hidden)
    np.save(a_name, a)
    b_name = directory + os.sep + "b_v" + str(n_examples) + "_h" + str(num_hidden)
    np.save(b_name, b)


def test_mnist(n_examples, num_hidden, epochs, learn_rate):
   # load data
    images, labels = load_mnist(n_examples, training = True)

   # train one layer of RBM
    w, a, b  = rbm(images, num_hidden, learn_rate, epochs, batchsize = 100)

   # save all weights
    print("Saving weights...")
    save_weights(w, a, b, "Output", n_examples, num_hidden)
   
   # try to reconstruct some test set images
    print("Generating and saving the reconstructed images...")
    images, labels = load_mnist(10, training = False)
    for i in range(10):
        data = images[i]
        save_mnist_image(data, "Output", str(i) + "original.png")
        data = reconstruct(data, w, a, b)
        save_mnist_image(data, "Output", str(i) + "reconstructed.png")
    print("Done!")


if __name__ == '__main__':
    if len(sys.argv) == 5:
      # user provided the params
        n_examples = int(sys.argv[1])
        num_hidden = int(sys.argv[2])
        epochs = int(sys.argv[3])
        learn_rate = float(sys.argv[4])
      # run!
        test_mnist(n_examples, num_hidden, epochs, learn_rate)
    else:
        test_mnist(1000, 100, 50, 0.1)


IndexError: tuple index out of range

In [22]:
def load_mnist(n_examples, training = True):
    return np.array([[1],[1]])
    n_train = n_examples  # The size of training set
    # Split dataset into training set (60000) and testing set (10000)
    data_train = mnist.data[:n_train]
    target_train = mnist.target[:n_train]
    data_test = mnist.data[n_train:]
    target_test = mnist.target[n_train:]
    return (data_train.astype(np.float32), target_train.astype(np.float32),
            data_test.astype(np.float32), target_test.astype(np.float32))